# NFL Fantasy Draft Optimizer
#### Developed by Dan McDonough
#### February 8, 2021

The purpose of this notebook is to help make real-time decisions on draft night.

I don't have a whole lot of NFL domain knowledge, so I didn't want to spend time trying to build a player projection model. Instead, I wanted to try to build competitive advantage by being able to maximize the number of projected points drafted, with a given set of projections taken as ground truth.

Unfortunately, this tool did not lead me to a fantasy championship, but it was still a fun thought experiment nonetheless.

### 1. Scrape Player Rankings

To start, I scraped the player rankings from ESPN. This is used later on to simulate the draft selections of other players in the league.

In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import random
import timeit
import numpy as np

In [2]:
def scrape_rankings():
    url = 'https://www.espn.com/fantasy/football/story/_/id/29083378/mike-clay-updated-2020-fantasy-football-rankings-non-ppr'
    response = requests.get(url) 
    soup = BeautifulSoup(response.text, "html.parser")
    
    rank = list(range(1,301))
    player = []
    position = []
    team = []
    bye = []
    posrank = []
    data = str(soup.findAll('p')[2]).split('<a href=')[1:]
    for row in data:
        player.append(row.split('>')[1].split('<')[0])
        position.append(row.split('-- ')[1].split(',')[0][0])
        team.append(row.split(', ')[1].split(' ')[0])
        bye.append(row.split(': ')[1].split(')')[0])
        tempval = row.split('-- ')[1].split(',')[0]
        if len(tempval) == 2:
            posrank.append(tempval[-1])
        elif len(tempval) == 3:
            if tempval[0] == 'K':
                posrank.append(tempval[-2:])
            else:
                posrank.append(tempval[-1])
        elif len(tempval) == 4:
            if tempval[0] == 'D':
                posrank.append(tempval[-1])
            else:
                posrank.append(tempval[-2:])
        elif len(tempval) == 5:
            if tempval[0] == 'D':
                posrank.append(tempval[-2:])
            else:
                posrank.append(tempval[-3:])

    player_rankings = pd.DataFrame({'rank':rank,'player':player,'position':position,'team':team,'bye':bye,'posrank':posrank})
    
    return player_rankings

In [3]:
player_rankings = scrape_rankings()
player_rankings[0:10]

# save data in case HTML changes
# player_rankings.to_csv('rankings_df_2020.csv',index=False)
# player_rankings = pd.read_csv('rankings_df_2020.csv')

,rank,player,position,team,bye,posrank
0,1,Christian McCaffrey,R,CAR,13,1
1,2,Saquon Barkley,R,NYG,11,2
2,3,Ezekiel Elliott,R,DAL,10,3
3,4,Dalvin Cook,R,MIN,7,4
4,5,Alvin Kamara,R,NO,6,5
5,6,Michael Thomas,W,NO,6,1
6,7,Derrick Henry,R,TEN,7,6
7,8,Clyde Edwards-Helaire,R,KC,10,7
8,9,Miles Sanders,R,PHI,9,8
9,10,Josh Jacobs,R,LV,6,9


### 2. Scrape Projections

Next I need to scrape a source of fantasy projections. For my purposes, I am going to treat these projections as truth, and determine how to optimally draft a lineup that maximizes the number of projected fantasy points.

#### QB Projections

In [4]:
def scrape_qb():
    url = 'https://www.fantasypros.com/nfl/projections/qb.php?week=draft'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    player_stat = []
    attempts = []
    completions = []
    yards = []
    touchdowns = []
    interceptions = []
    rush_attempts = []
    rush_yards = []
    rush_tds = []
    fumbles = []

    for i in range(30):
        name = len(str(soup.findAll('td')[4+11*i].findAll('a')[0]).split('>')[1])
        player_stat.append(str(soup.findAll('td')[4+11*i].findAll('a')[0]).split('>')[1][0:name-3])
        attempts.append(float(str(soup.findAll('td')[5+11*i]).split('>')[1].split('<')[0]))
        completions.append(float((str(soup.findAll('td')[6+11*i]).split('>')[1].split('<')[0])))
        yards.append(float(str(soup.findAll('td')[7+11*i]).split('>')[1].split('<')[0].split(',')[0]+str(soup.findAll('td')[7+11*i]).split('>')[1].split('<')[0].split(',')[1]))
        touchdowns.append(float((str(soup.findAll('td')[8+11*i]).split('>')[1].split('<')[0])))
        interceptions.append(float((str(soup.findAll('td')[9+11*i]).split('>')[1].split('<')[0])))
        rush_attempts.append(float((str(soup.findAll('td')[10+11*i]).split('>')[1].split('<')[0])))
        rush_yards.append(float((str(soup.findAll('td')[11+11*i]).split('>')[1].split('<')[0])))
        rush_tds.append(float((str(soup.findAll('td')[12+11*i]).split('>')[1].split('<')[0])))
        fumbles.append(float((str(soup.findAll('td')[13+11*i]).split('>')[1].split('<')[0])))
    
    qb_projections = pd.DataFrame({'player_stat':player_stat,'attempts':attempts,'completions':completions,'yards':yards,
                             'touchdowns':touchdowns,'interceptions':interceptions,'rush_attempts':rush_attempts,
                             'rush_yards':rush_yards,'rush_tds':rush_tds,'fumbles':fumbles})

    return qb_projections

In [5]:
qb_projections = scrape_qb()
qb_projections[0:10]

# save data in case HTML changes
# qb_projections.to_csv('qb_df_2020.csv',index=False)
# qb_projections = pd.read_csv('qb_df_2020.csv')

,player_stat,attempts,completions,yards,touchdowns,interceptions,rush_attempts,rush_yards,rush_tds,fumbles
0,Lamar Jackson,469.0,301.3,3484.1,28.9,11.6,159.7,955.4,5.9,3.4
1,Patrick Mahomes II,568.3,371.3,4574.9,34.5,10.7,57.5,298.5,3.0,2.1
2,Dak Prescott,573.7,373.2,4505.4,28.6,12.1,60.6,258.5,3.7,2.8
3,Deshaun Watson,526.6,346.3,4019.1,26.3,12.7,91.4,496.8,4.6,2.6
4,Russell Wilson,492.0,321.0,3900.6,29.5,8.5,72.5,378.7,2.3,2.3
5,Kyler Murray,560.4,361.9,3967.3,25.0,12.7,87.3,475.1,3.3,2.4
6,Josh Allen,514.4,305.1,3511.5,21.0,12.8,110.9,610.4,6.5,3.7
7,Matt Ryan,624.5,413.8,4676.4,28.1,13.3,34.4,132.8,1.3,3.5
8,Tom Brady,571.7,361.1,4334.6,28.9,10.3,24.4,36.6,1.4,1.6
9,Drew Brees,534.0,379.9,4183.4,30.4,9.8,23.0,25.9,1.1,1.4


Next, I need to standardize names across data sets

In [6]:
def check_players(x, position):
    print("In projections, but not in rankings: ")
    for index, player in enumerate(x['player_stat']):
        if player not in player_rankings[player_rankings['position']==position]['player'].unique():        
            print(index,' ',player)
    test_df = player_rankings.merge(x,how='left',left_on='player',right_on='player_stat')
    missing_df = test_df[(test_df['position']==position)&(test_df['player_stat'].isna())]
    if len(missing_df) > 0:
        print('\nIn rankings, but not in projections:')
        return missing_df

In [7]:
check_players(qb_projections, 'Q')

In projections, but not in rankings: 
1   Patrick Mahomes II
29   Dwayne Haskins

In rankings, but not in projections:


,rank,player,position,team,bye,posrank,player_stat,attempts,completions,yards,touchdowns,interceptions,rush_attempts,rush_yards,rush_tds,fumbles
34,35,Patrick Mahomes,Q,KC,10,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267,268,Dwayne Haskins Jr.,Q,WAS,8,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
268,269,Tyrod Taylor,Q,LAC,10,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
269,270,Tua Tagovailoa,Q,MIA,11,31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
qb_projections.loc[1,'player_stat'] = 'Patrick Mahomes'
qb_projections.loc[29,'player_stat'] = 'Dwayne Haskins Jr.'

In [9]:
check_players(qb_projections, 'Q')

In projections, but not in rankings: 

In rankings, but not in projections:


,rank,player,position,team,bye,posrank,player_stat,attempts,completions,yards,touchdowns,interceptions,rush_attempts,rush_yards,rush_tds,fumbles
268,269,Tyrod Taylor,Q,LAC,10,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
269,270,Tua Tagovailoa,Q,MIA,11,31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Next I will add my league's scoring

In [10]:
scoring_dict = {}
scoring_dict['passing_yards_score'] = 0.04
scoring_dict['passing_TD_score'] = 4
scoring_dict['passing_interception_score'] = -2
scoring_dict['rushing_yards_score'] = 0.1
scoring_dict['rushing_TD_score'] = 6
scoring_dict['offense_fumble_score'] = -2
scoring_dict['receptions_score'] = 0
scoring_dict['receiving_yards_score'] = 0.1
scoring_dict['receiving_TD_score'] = 6
scoring_dict['sack_score'] = 1
scoring_dict['defense_interception_score'] = 2
scoring_dict['defense_fumble_score'] = 2
scoring_dict['defense_TD_score'] = 6
scoring_dict['safety_score'] = 2
scoring_dict['points_against_score'] = -2/7
scoring_dict['fg_made_score'] = 4
scoring_dict['fg_miss_score'] = -1
scoring_dict['xp_made_score'] = 1
scoring_dict['xp_miss_score'] = -1

In [11]:
def qb_scoring(x, scoring_dict):
    points = x['yards'] * scoring_dict['passing_yards_score'] + x['touchdowns'] * scoring_dict['passing_TD_score'] +\
        x['interceptions'] * scoring_dict['passing_interception_score'] +\
        x['rush_yards'] * scoring_dict['rushing_yards_score'] + x['rush_tds'] * scoring_dict['rushing_TD_score'] +\
        x['fumbles'] * scoring_dict['offense_fumble_score']
    
    return points
    
qb_projections['points'] = qb_projections.apply(qb_scoring, args =(scoring_dict, ), axis=1)

In [12]:
qb_projections.head(1)

,player_stat,attempts,completions,yards,touchdowns,interceptions,rush_attempts,rush_yards,rush_tds,fumbles,points
0,Lamar Jackson,469.0,301.3,3484.1,28.9,11.6,159.7,955.4,5.9,3.4,355.904


#### RB Projections

In [13]:
def scrape_rb():
    url = 'https://www.fantasypros.com/nfl/projections/rb.php?week=draft'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    player_stat = []
    rush_attempts = []
    rush_yards = []
    rush_tds = []
    receptions = []
    receive_yards = []
    receive_tds = []
    fumbles = []
    for i in range(100):
        name = len(str(soup.findAll('td')[4+9*i].findAll('a')[0]).split('>')[1])
        player_stat.append(str(soup.findAll('td')[4+9*i].findAll('a')[0]).split('>')[1][0:name-3])
        rush_attempts.append(float((str(soup.findAll('td')[5+9*i]).split('>')[1].split('<')[0])))
        if len(str(soup.findAll('td')[6+9*i]).split('>')[1].split('<')[0]) < 6:
            rush_yards.append(float((str(soup.findAll('td')[6+9*i]).split('>')[1].split('<')[0])))
        else:    
            rush_yards.append(float((str(soup.findAll('td')[6+9*i]).split('>')[1].split('<')[0]).split(',')[0]+(str(soup.findAll('td')[6+9*i]).split('>')[1].split('<')[0]).split(',')[1]))
        rush_tds.append(float((str(soup.findAll('td')[7+9*i]).split('>')[1].split('<')[0])))
        receptions.append(float((str(soup.findAll('td')[8+9*i]).split('>')[1].split('<')[0])))
        receive_yards.append(float((str(soup.findAll('td')[9+9*i]).split('>')[1].split('<')[0])))
        receive_tds.append(float((str(soup.findAll('td')[10+9*i]).split('>')[1].split('<')[0])))
        fumbles.append(float((str(soup.findAll('td')[11+9*i]).split('>')[1].split('<')[0])))

    rb_projections = pd.DataFrame({'player_stat':player_stat,'rush_attempts':rush_attempts,'rush_yards':rush_yards,
                                   'rush_tds':rush_tds,'receptions':receptions,'receive_yards':receive_yards,
                                   'receive_tds':receive_tds,'fumbles':fumbles})
    
    return rb_projections

In [14]:
rb_projections = scrape_rb()
rb_projections[0:10]

# save data in case HTML changes
# rb_projections.to_csv('rb_df_2020.csv',index=False)
# rb_projections = pd.read_csv('rb_df_2020.csv')

,player_stat,rush_attempts,rush_yards,rush_tds,receptions,receive_yards,receive_tds,fumbles
0,Christian McCaffrey,249.0,1121.0,8.5,100.3,833.5,4.3,1.8
1,Ezekiel Elliott,286.0,1271.0,9.7,59.4,471.0,2.5,2.3
2,Saquon Barkley,269.3,1218.8,8.9,64.9,523.8,2.6,1.3
3,Derrick Henry,296.8,1462.4,12.0,22.3,189.7,1.1,2.0
4,Dalvin Cook,259.2,1138.1,9.8,59.7,526.2,1.8,2.4
5,Alvin Kamara,199.8,915.2,7.9,82.7,607.0,2.9,1.6
6,Clyde Edwards-Helaire,207.8,938.0,8.2,51.7,454.3,3.2,1.9
7,Nick Chubb,276.6,1264.7,9.1,25.2,195.4,1.1,1.8
8,Josh Jacobs,269.6,1215.1,9.2,30.9,239.8,0.9,1.7
9,Joe Mixon,273.0,1183.4,7.5,39.6,305.4,1.7,1.5


Next, I need to standardize names across data sets

In [15]:
check_players(rb_projections, 'R')

In projections, but not in rankings: 
18   Melvin Gordon III
42   Duke Johnson Jr.
47   Darrell Henderson
60   Adrian Peterson
83   Kyle Juszczyk
85   Reggie Bonnafon
88   Ryan Nall
89   Darwin Thompson
90   Corey Clement
91   Mike Boone
92   Dare Ogunbowale
93   Ty Montgomery
94   Malcolm Perry
95   T.J. Yeldon
96   Rico Dowdle
97   Eno Benjamin
98   Andy Janovich
99   Lynn Bowden Jr.

In rankings, but not in projections:


,rank,player,position,team,bye,posrank,player_stat,rush_attempts,rush_yards,rush_tds,receptions,receive_yards,receive_tds,fumbles
45,46,Melvin Gordon,R,DEN,8,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,70,Mark Ingram II,R,BAL,8,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,104,Darrell Henderson Jr.,R,LAR,9,42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,128,Duke Johnson,R,HOU,8,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
234,235,Ke'Shawn Vaughn,R,TB,13,72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
249,250,Wayne Gallman,R,NYG,11,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
279,280,Trayveon Williams,R,CIN,9,87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
282,283,Bo Scarbrough,R,DET,5,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
rb_projections.loc[18,'player_stat'] = 'Melvin Gordon'
rb_projections.loc[42,'player_stat'] = 'Duke Johnson'
rb_projections.loc[47,'player_stat'] = 'Darrell Henderson Jr.'

In [17]:
check_players(rb_projections, 'R')

In projections, but not in rankings: 
60   Adrian Peterson
83   Kyle Juszczyk
85   Reggie Bonnafon
88   Ryan Nall
89   Darwin Thompson
90   Corey Clement
91   Mike Boone
92   Dare Ogunbowale
93   Ty Montgomery
94   Malcolm Perry
95   T.J. Yeldon
96   Rico Dowdle
97   Eno Benjamin
98   Andy Janovich
99   Lynn Bowden Jr.

In rankings, but not in projections:


,rank,player,position,team,bye,posrank,player_stat,rush_attempts,rush_yards,rush_tds,receptions,receive_yards,receive_tds,fumbles
69,70,Mark Ingram II,R,BAL,8,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
234,235,Ke'Shawn Vaughn,R,TB,13,72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
249,250,Wayne Gallman,R,NYG,11,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
279,280,Trayveon Williams,R,CIN,9,87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
282,283,Bo Scarbrough,R,DET,5,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
def rb_scoring(x, scoring_dict):
    points = x['rush_yards'] * scoring_dict['rushing_yards_score'] + x['rush_tds'] * scoring_dict['rushing_TD_score'] +\
    x['receptions'] * scoring_dict['receptions_score'] + x['receive_yards'] * scoring_dict['receiving_yards_score'] +\
    x['receive_tds'] * scoring_dict['receiving_TD_score'] + x['fumbles'] * scoring_dict['offense_fumble_score']
    
    return points
    
rb_projections['points'] = rb_projections.apply(rb_scoring, args= (scoring_dict,), axis=1)

In [19]:
rb_projections.head(1)

,player_stat,rush_attempts,rush_yards,rush_tds,receptions,receive_yards,receive_tds,fumbles,points
0,Christian McCaffrey,249.0,1121.0,8.5,100.3,833.5,4.3,1.8,268.65


### WR Projections

In [20]:
def scrape_wr():
    url = 'https://www.fantasypros.com/nfl/projections/wr.php?week=draft'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    player_stat = []
    receptions = []
    receive_yards = []
    receive_tds = []
    rush_attempts = []
    rush_yards = []
    rush_tds = []
    fumbles = []
    for i in range(100):
        name = len(str(soup.findAll('td')[4+9*i].findAll('a')[0]).split('>')[1])
        player_stat.append(str(soup.findAll('td')[4+9*i].findAll('a')[0]).split('>')[1][0:name-3])
        receptions.append(float((str(soup.findAll('td')[5+9*i]).split('>')[1].split('<')[0])))
        if len(str(soup.findAll('td')[6+9*i]).split('>')[1].split('<')[0]) < 6:
            receive_yards.append(float((str(soup.findAll('td')[6+9*i]).split('>')[1].split('<')[0])))
        else:
            receive_yards.append(float((str(soup.findAll('td')[6+9*i]).split('>')[1].split('<')[0].split(',')[0]+str(soup.findAll('td')[6+9*i]).split('>')[1].split('<')[0].split(',')[1])))
        receive_tds.append(float((str(soup.findAll('td')[7+9*i]).split('>')[1].split('<')[0])))
        rush_attempts.append(float((str(soup.findAll('td')[8+9*i]).split('>')[1].split('<')[0])))
        rush_yards.append(float((str(soup.findAll('td')[9+9*i]).split('>')[1].split('<')[0])))
        rush_tds.append(float((str(soup.findAll('td')[10+9*i]).split('>')[1].split('<')[0])))
        fumbles.append(float((str(soup.findAll('td')[11+9*i]).split('>')[1].split('<')[0])))

    wr_projections = pd.DataFrame({'player_stat':player_stat,'receptions':receptions,'receive_yards':receive_yards,
                                   'receive_tds':receive_tds,'rush_attempts':rush_attempts,'rush_yards':rush_yards,
                                   'rush_tds':rush_tds,'fumbles':fumbles})
    
    return wr_projections

In [21]:
wr_projections = scrape_wr()
wr_projections[0:10]

# save data in case HTML changes
# wr_projections.to_csv('wr_df_2020.csv',index=False)
# wr_projections = pd.read_csv('wr_df_2020.csv')

,player_stat,receptions,receive_yards,receive_tds,rush_attempts,rush_yards,rush_tds,fumbles
0,Michael Thomas,126.3,1505.2,9.0,0.3,0.9,0.0,0.5
1,Davante Adams,103.8,1306.5,9.7,0.0,0.0,0.0,0.5
2,Julio Jones,100.0,1410.5,6.8,1.0,4.2,0.0,0.5
3,Tyreek Hill,81.5,1181.7,8.4,10.7,66.5,0.5,0.4
4,Chris Godwin,83.3,1195.6,7.7,1.1,6.6,0.1,0.5
5,DeAndre Hopkins,94.7,1171.0,7.6,1.3,7.6,0.0,0.5
6,Kenny Golladay,71.4,1148.6,7.9,0.3,1.7,0.0,0.5
7,Mike Evans,73.2,1142.8,7.6,0.5,3.1,0.0,0.5
8,D.J. Moore,86.1,1170.9,5.4,5.8,49.2,0.1,1.0
9,Amari Cooper,77.1,1096.2,7.1,0.8,6.5,0.0,0.5


Next, I need to standardize names across data sets

In [22]:
check_players(wr_projections, 'W')

In projections, but not in rankings: 
8   D.J. Moore
21   D.K. Metcalf
27   D.J. Chark Jr.
59   Steven Sims
80   John Ross
85   Dontrelle Inman
86   Josh Gordon
91   Kendrick Bourne
94   Tre'Quan Smith
96   Olabisi Johnson
97   KJ Hamler
99   Phillip Dorsett II

In rankings, but not in projections:


,rank,player,position,team,bye,posrank,player_stat,receptions,receive_yards,receive_tds,rush_attempts,rush_yards,rush_tds,fumbles
24,25,DJ Moore,W,CAR,13,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,53,DK Metcalf,W,SEA,6,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,55,DJ Chark Jr.,W,JAC,7,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,194,Steven Sims Jr.,W,WAS,8,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213,214,Antonio Brown,W,FA,-,76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
225,226,Scotty Miller,W,TB,13,82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227,228,Trent Taylor,W,SF,11,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
228,229,Devin Duvernay,W,BAL,8,85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259,260,Ted Ginn Jr.,W,CHI,11,93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
261,262,John Ross III,W,CIN,9,95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
wr_projections.loc[8,'player_stat'] = 'DJ Moore'
wr_projections.loc[21,'player_stat'] = 'DK Metcalf'
wr_projections.loc[27,'player_stat'] = 'DJ Chark Jr.'
wr_projections.loc[59,'player_stat'] = 'Steven Sims Jr.'
wr_projections.loc[80,'player_stat'] = 'John Ross III'

In [24]:
check_players(wr_projections, 'W')

In projections, but not in rankings: 
85   Dontrelle Inman
86   Josh Gordon
91   Kendrick Bourne
94   Tre'Quan Smith
96   Olabisi Johnson
97   KJ Hamler
99   Phillip Dorsett II

In rankings, but not in projections:


,rank,player,position,team,bye,posrank,player_stat,receptions,receive_yards,receive_tds,rush_attempts,rush_yards,rush_tds,fumbles
213,214,Antonio Brown,W,FA,-,76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
225,226,Scotty Miller,W,TB,13,82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227,228,Trent Taylor,W,SF,11,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
228,229,Devin Duvernay,W,BAL,8,85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259,260,Ted Ginn Jr.,W,CHI,11,93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262,263,Gunner Olszewski,W,NE,6,96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
271,272,Jalen Guyton,W,LAC,10,98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273,274,Chris Hogan,W,NYJ,11,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
274,275,Jakobi Meyers,W,NE,6,101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
283,284,Demarcus Robinson,W,KC,10,102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
def wr_scoring(x, scoring_dict):
    points = x['rush_yards'] * scoring_dict['rushing_yards_score'] + x['rush_tds'] * scoring_dict['rushing_TD_score'] +\
    x['receptions'] * scoring_dict['receptions_score'] + x['receive_yards'] * scoring_dict['receiving_yards_score'] +\
    x['receive_tds'] * scoring_dict['receiving_TD_score'] + x['fumbles'] * scoring_dict['offense_fumble_score']
    
    return points
    
wr_projections['points'] = wr_projections.apply(wr_scoring, args= (scoring_dict,), axis=1)

In [26]:
wr_projections.head(1)

,player_stat,receptions,receive_yards,receive_tds,rush_attempts,rush_yards,rush_tds,fumbles,points
0,Michael Thomas,126.3,1505.2,9.0,0.3,0.9,0.0,0.5,203.61


### TE Projections

In [27]:
def scrape_te():
    url = 'https://www.fantasypros.com/nfl/projections/te.php?week=draft'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    player_stat = []
    receptions = []
    receive_yards = []
    receive_tds = []
    rush_attempts = []
    rush_yards = []
    rush_tds = []
    fumbles = []
    for i in range(80):
        name = len(str(soup.findAll('td')[3+6*i].findAll('a')[0]).split('>')[1])
        player_stat.append(str(soup.findAll('td')[3+6*i].findAll('a')[0]).split('>')[1][0:name-3])
        receptions.append(float((str(soup.findAll('td')[4+6*i]).split('>')[1].split('<')[0])))
        if len(str(soup.findAll('td')[5+6*i]).split('>')[1].split('<')[0]) < 6:
            receive_yards.append(float((str(soup.findAll('td')[5+6*i]).split('>')[1].split('<')[0])))
        else:
            receive_yards.append(float(str(soup.findAll('td')[5+6*i]).split('>')[1].split('<')[0].split(',')[0]+str(soup.findAll('td')[5+6*i]).split('>')[1].split('<')[0].split(',')[1]))
        receive_tds.append(float((str(soup.findAll('td')[6+6*i]).split('>')[1].split('<')[0])))
        fumbles.append(float((str(soup.findAll('td')[7+6*i]).split('>')[1].split('<')[0])))

    te_projections = pd.DataFrame({'player_stat':player_stat,'receptions':receptions,'receive_yards':receive_yards,
                                   'receive_tds':receive_tds,'fumbles':fumbles})
    
    return te_projections

In [28]:
te_projections = scrape_te()
te_projections[0:10]

# save data in case HTML changes
# te_projections.to_csv('te_df_2020.csv',index=False)
# te_projections = pd.read_csv('te_df_2020.csv')

,player_stat,receptions,receive_yards,receive_tds,fumbles
0,Travis Kelce,92.8,1139.2,8.2,0.5
1,George Kittle,87.1,1069.9,6.4,0.5
2,Mark Andrews,63.0,840.0,7.6,0.5
3,Zach Ertz,83.7,875.0,6.6,0.5
4,Darren Waller,74.1,878.6,4.7,0.5
5,Evan Engram,64.9,736.7,5.0,0.4
6,Jared Cook,49.4,667.6,5.8,0.1
7,Hunter Henry,59.1,699.7,5.1,0.5
8,Tyler Higbee,63.4,712.7,4.5,0.4
9,Rob Gronkowski,47.2,656.3,4.6,0.4


Next, I need to standardize names across data sets

In [29]:
check_players(te_projections, 'T')

In projections, but not in rankings: 
14   Chris Herndon IV
28   Darren Fells
30   Jace Sternberger
31   C.J. Uzomah
32   David Njoku
34   Nick Boyle
36   Devin Asiasi
37   Vance McDonald
40   Trey Burton
41   Anthony Firkser
42   Cole Kmet
43   Jordan Reed
44   Logan Thomas
45   Ryan Griffin
46   James O'Shaughnessy
47   Jesse James
48   Cameron Brate
49   Jacob Hollister
51   Maxx Williams
52   Robert Tonyan
53   Mo Alie-Cox
54   Tyler Kroft
55   Josh Hill
56   Jeremy Sprinkle
57   Jason Witten
58   Jaeden Graham
59   Garrett Griffin
60   Harrison Bryant
61   Ricky Seals-Jones
62   Foster Moreau
63   Adam Shaheen
64   MyCole Pruitt
65   Drew Sample
66   Marcedes Lewis
67   Andrew Beck
68   Jordan Thomas
69   Jesper Horsted
70   Nick Vannett
71   Levine Toilolo
72   Demetrius Harris
73   Dalton Schultz
74   Josiah Deguara
75   Virgil Green
76   Charlie Woerner
77   Adam Trautman
78   Jerell Adams
79   Ryan Izzo

In rankings, but not in projections:


,rank,player,position,team,bye,posrank,player_stat,receptions,receive_yards,receive_tds,fumbles
134,135,Chris Herndon,T,NYJ,11,15,NaN,NaN,NaN,NaN,NaN


In [30]:
te_projections.loc[14,'player_stat'] = 'Chris Herndon'

In [31]:
check_players(te_projections, 'T')

In projections, but not in rankings: 
28   Darren Fells
30   Jace Sternberger
31   C.J. Uzomah
32   David Njoku
34   Nick Boyle
36   Devin Asiasi
37   Vance McDonald
40   Trey Burton
41   Anthony Firkser
42   Cole Kmet
43   Jordan Reed
44   Logan Thomas
45   Ryan Griffin
46   James O'Shaughnessy
47   Jesse James
48   Cameron Brate
49   Jacob Hollister
51   Maxx Williams
52   Robert Tonyan
53   Mo Alie-Cox
54   Tyler Kroft
55   Josh Hill
56   Jeremy Sprinkle
57   Jason Witten
58   Jaeden Graham
59   Garrett Griffin
60   Harrison Bryant
61   Ricky Seals-Jones
62   Foster Moreau
63   Adam Shaheen
64   MyCole Pruitt
65   Drew Sample
66   Marcedes Lewis
67   Andrew Beck
68   Jordan Thomas
69   Jesper Horsted
70   Nick Vannett
71   Levine Toilolo
72   Demetrius Harris
73   Dalton Schultz
74   Josiah Deguara
75   Virgil Green
76   Charlie Woerner
77   Adam Trautman
78   Jerell Adams
79   Ryan Izzo


In [32]:
def te_scoring(x, scoring_dict):
    points = x['receptions'] * scoring_dict['receptions_score'] +\
    x['receive_yards'] * scoring_dict['receiving_yards_score'] + x['receive_tds'] * scoring_dict['receiving_TD_score'] +\
    x['fumbles'] * scoring_dict['offense_fumble_score']
    
    return points
    
te_projections['points'] = te_projections.apply(te_scoring, args= (scoring_dict,), axis=1)

In [33]:
te_projections.head(1)

,player_stat,receptions,receive_yards,receive_tds,fumbles,points
0,Travis Kelce,92.8,1139.2,8.2,0.5,162.12


### D/ST Projections

In [34]:
def scrape_dst():
    url = 'https://www.fantasypros.com/nfl/projections/dst.php?week=draft'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    player_stat = []
    sack = []
    interception = []
    fumble_recovered = []
    touchdowns = []
    safety = []
    points_against = []
    yards_against = []

    for i in range(32):
        name = len(str(soup.findAll('td')[0+10*i].findAll('a')[0]).split('>')[1])
        player_stat.append(str(soup.findAll('td')[0+10*i].findAll('a')[0]).split('>')[1][0:name-3])
        sack.append(float((str(soup.findAll('td')[1+10*i]).split('>')[1].split('<')[0])))
        interception.append(float((str(soup.findAll('td')[2+10*i]).split('>')[1].split('<')[0])))
        fumble_recovered.append(float((str(soup.findAll('td')[3+10*i]).split('>')[1].split('<')[0])))
        touchdowns.append(float((str(soup.findAll('td')[5+10*i]).split('>')[1].split('<')[0])))
        safety.append(float((str(soup.findAll('td')[6+10*i]).split('>')[1].split('<')[0])))
        points_against.append(float((str(soup.findAll('td')[7+10*i]).split('>')[1].split('<')[0])))
        yards_against.append((str(soup.findAll('td')[8+10*i]).split('>')[1].split('<')[0]))

    dst_projections = pd.DataFrame({'player_stat':player_stat,'sack':sack,'interception':interception,
                                   'fumble_recovered':fumble_recovered,'touchdowns':touchdowns,
                                   'safety':safety,'points_against':points_against,'yards_against':yards_against})
    
    return dst_projections

In [35]:
dst_projections = scrape_dst()
dst_projections[0:10]

# save data in case HTML changes
# dst_projections.to_csv('dst_df_2020.csv',index=False)
# dst_projections = pd.read_csv('dst_df_2020.csv')

,player_stat,sack,interception,fumble_recovered,touchdowns,safety,points_against,yards_against
0,Pittsburgh Steelers,47.3,14.4,12.8,2.8,0.6,317.6,"5,129.0"
1,San Francisco 49ers,45.7,14.0,12.5,2.9,0.2,320.0,"5,072.6"
2,Los Angeles Rams,44.6,13.9,11.0,3.0,0.5,353.1,"5,405.7"
3,New England Patriots,41.3,15.4,10.3,2.9,0.8,319.1,"5,207.5"
4,Baltimore Ravens,39.9,14.6,10.4,3.1,0.5,306.4,"5,111.7"
5,Minnesota Vikings,43.1,13.4,11.6,2.4,0.9,332.6,"5,416.7"
6,Philadelphia Eagles,40.3,14.4,10.6,2.8,0.5,359.8,"5,569.0"
7,Kansas City Chiefs,41.7,14.5,9.4,3.0,0.2,338.3,"5,620.3"
8,New Orleans Saints,43.0,12.7,10.9,2.7,0.5,349.7,"5,519.5"
9,Buffalo Bills,40.5,13.9,10.6,2.7,0.5,304.6,"4,978.6"


Next, I need to standardize names across data sets

In [36]:
check_players(dst_projections, 'D')

In projections, but not in rankings: 
0   Pittsburgh Steelers
1   San Francisco 49ers
2   Los Angeles Rams
3   New England Patriots
4   Baltimore Ravens
5   Minnesota Vikings
6   Philadelphia Eagles
7   Kansas City Chiefs
8   New Orleans Saints
9   Buffalo Bills
10   Tampa Bay Buccaneers
11   Indianapolis Colts
12   Seattle Seahawks
13   Cleveland Browns
14   Green Bay Packers
15   Dallas Cowboys
16   Carolina Panthers
17   Washington Football Team
18   Denver Broncos
19   Tennessee Titans
20   Houston Texans
21   Arizona Cardinals
22   Jacksonville Jaguars
23   New York Jets
24   Chicago Bears
25   New York Giants
26   Cincinnati Bengals
27   Las Vegas Raiders
28   Detroit Lions
29   Los Angeles Chargers
30   Atlanta Falcons
31   Miami Dolphins

In rankings, but not in projections:


,rank,player,position,team,bye,posrank,player_stat,sack,interception,fumble_recovered,touchdowns,safety,points_against,yards_against
168,169,Steelers D/ST,D,PIT,8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169,170,Bills D/ST,D,BUF,11,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,171,49ers D/ST,D,SF,11,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,172,Ravens D/ST,D,BAL,8,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,173,Patriots D/ST,D,NE,6,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173,174,Colts D/ST,D,IND,7,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174,175,Bears D/ST,D,CHI,11,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175,176,Vikings D/ST,D,MIN,7,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176,177,Broncos D/ST,D,DEN,8,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177,178,Saints D/ST,D,NO,6,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
dst_projections['player_stat'] = dst_projections['player_stat'].apply(lambda x: x.split(' ')[-1] + ' D/ST')

In [38]:
check_players(dst_projections, 'D')

In projections, but not in rankings: 
14   Packers D/ST
16   Panthers D/ST
17   Team D/ST
20   Texans D/ST
21   Cardinals D/ST
22   Jaguars D/ST
23   Jets D/ST
25   Giants D/ST
27   Raiders D/ST
30   Falcons D/ST


In [39]:
def dst_scoring(x, scoring_dict):
    points = x['sack'] * scoring_dict['sack_score'] + x['interception'] * scoring_dict['defense_interception_score'] +\
    x['fumble_recovered'] * scoring_dict['defense_fumble_score'] + x['touchdowns'] * scoring_dict['defense_TD_score'] +\
    x['safety'] * scoring_dict['safety_score'] + x['points_against'] * scoring_dict['points_against_score'] + 7
    
    return points
    
dst_projections['points'] = dst_projections.apply(dst_scoring, args= (scoring_dict,), axis=1)

In [40]:
dst_projections.head(1)

,player_stat,sack,interception,fumble_recovered,touchdowns,safety,points_against,yards_against,points
0,Steelers D/ST,47.3,14.4,12.8,2.8,0.6,317.6,"5,129.0",35.957143


### K Projections

In [41]:
def scrape_k():
    url = 'https://www.fantasypros.com/nfl/projections/k.php?week=draft'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    player_stat = []
    fgm = []
    fga = []
    xp = []



    for i in range(32):
        name = len(str(soup.findAll('td')[0+5*i].findAll('a')[0]).split('>')[1])
        player_stat.append(str(soup.findAll('td')[0+5*i].findAll('a')[0]).split('>')[1][0:name-3])
        fgm.append(float((str(soup.findAll('td')[1+5*i]).split('>')[1].split('<')[0])))
        fga.append(float((str(soup.findAll('td')[2+5*i]).split('>')[1].split('<')[0])))
        xp.append(float((str(soup.findAll('td')[3+5*i]).split('>')[1].split('<')[0])))

    k_projections = pd.DataFrame({'player_stat':player_stat,'fgm':fgm,'fga':fga,'xp':xp})
    
    return k_projections

In [42]:
k_projections = scrape_k()
k_projections[0:10]

# save data in case HTML changes
# k_projections.to_csv('k_df_2020.csv',index=False)
# k_projections = pd.read_csv('k_df_2020.csv')

,player_stat,fgm,fga,xp
0,Harrison Butker,27.9,33.0,47.6
1,Wil Lutz,27.6,32.4,46.2
2,Justin Tucker,26.6,30.4,46.7
3,Robbie Gould,26.7,31.4,41.0
4,Greg Zuerlein,25.9,32.0,42.8
5,Matt Prater,25.9,31.3,36.4
6,Jason Myers,25.1,29.7,38.9
7,Dan Bailey,25.2,30.0,37.4
8,Younghoe Koo,25.5,30.3,36.5
9,Austin Seibert,26.1,30.8,34.5


Next, I need to standardize names across data sets

In [43]:
check_players(k_projections, 'K')

In projections, but not in rankings: 
13   Justin Rohrwasser
17   Ryan Succop
19   Nick Folk
20   Michael Badgley
22   Stephen Gostkowski
23   Randy Bullock
24   Josh Lambo
25   Jason Sanders
26   Tyler Bass
27   Joey Slye
28   Daniel Carlson
29   Graham Gano
30   Dustin Hopkins
31   Sam Ficken


In [44]:
def k_scoring(x, scoring_dict):
    points = x['fgm'] * scoring_dict['fg_made_score'] +\
        (x['fga'] - x['fgm']) * scoring_dict['fg_miss_score'] +\
        x['xp'] * scoring_dict['xp_made_score'] + x['xp'] * .05 * scoring_dict['xp_miss_score']
    
    return points
    
k_projections['points'] = k_projections.apply(k_scoring, args= (scoring_dict,), axis=1)

In [45]:
k_projections.head(1)

,player_stat,fgm,fga,xp,points
0,Harrison Butker,27.9,33.0,47.6,151.72


### Combine projections

In [46]:
all_projections = pd.concat([qb_projections[['player_stat','points']],rb_projections[['player_stat','points']],wr_projections[['player_stat','points']],
          te_projections[['player_stat','points']],dst_projections[['player_stat','points']],k_projections[['player_stat','points']]])

full_df = player_rankings.merge(all_projections,left_on='player',right_on='player_stat')
full_df.sort_values(by='points',ascending=False,inplace=True)
full_df.head()

,rank,player,position,team,bye,posrank,player_stat,points
33,34,Lamar Jackson,Q,BAL,8,1,Lamar Jackson,355.904
34,35,Patrick Mahomes,Q,KC,10,2,Patrick Mahomes,343.246
73,75,Dak Prescott,Q,DAL,10,5,Dak Prescott,312.866
71,73,Deshaun Watson,Q,HOU,8,3,Deshaun Watson,312.644
74,76,Russell Wilson,Q,SEA,6,6,Russell Wilson,304.094


### 3. Develop Model

Now that I have a combined dataframe - with rankings, projections, and fantasy scoring - I am going to define the functions I need for in-draft management.

First up are functions to add players to my team, drop players from the available player list, and find the best available players for at a position

In [47]:
def addplayer(my_team, current_df, name):
    nextplayer = current_df[current_df['player']==name]
    my_team = pd.concat([my_team,nextplayer])
    return my_team

def dropplayer(df, name):
    df = df[df['player']!=name]
    return df

def findplayer(my_team,df,position):
    test = df[(df['position']==position)]
    bye_list = list(my_team[(my_team['position']==position)]['bye'])
    for i in range(len(bye_list)):
        test = test[test['bye']!=bye_list[i]]
    return test[0:5]

Next I will initialize the dataframes for my team and the broader player list

In [48]:
draft_df = full_df.copy()
my_team = full_df.copy()
my_team = my_team[my_team['player']=='a']

Now I will define the function that simulates the draft.

My goal is to run a monte-carlo style simulation to determine which position/player I should take next in the draft.

For my team's selections, I randomly choose a position at each round and draft the best available player at that position.

I didn't want to guide the model too heavily, but there are a few constraints I put in place to keep the simulations realistic enough to be informative
<br> -- How many players can be taken at a single position before a certain round (e.g. multiple kickers early in the draft)
<br> -- Weightings for player projections based on my team's dpeth chart (e.g. only one QB scan score in a given week)
<br> -- Not drafting multiple players at the same position with the same bye week

For my opponents, I would assume that they randomly chose one of the top three best available players. I added this variability so that the model would not bank on specific players being available at specific spots later in the draft.

In [49]:
def simulate(my_team,draft_df,simulations):
    current_df = draft_df.copy()
    # I drafted 5th in our ten-person snake draft
    allpicks = [5,16,25,36,45,56,65,76,85,96,105,116,125,136,145,156]
    roundnum = len(my_team)
    mypicks = (np.array(allpicks)-roundnum*10)[roundnum:17]
    picks = pd.DataFrame(columns=['Rnd1','Rnd2','Rnd3','Rnd4','Rnd5','Rnd6','Rnd7','Rnd8','Rnd9','Rnd10','Rnd11','Rnd12','Rnd13','Rnd14','Rnd15','Rnd16','Pts'],index=range(simulations))
    for i in range(len(my_team)):
        picks.iloc[:,i] = my_team['position'][i]
    currpoints = 0
    runpoints = sum(my_team['points'])
    
    # these are the decay factors for how much of that player's season-long projections I can reasonably expect
    # to score in my lineup
    qb_dict = dict(zip(range(16),[.93,.07,0,0,0,0,0,0,0,0,0,0,0,0,0,0]))
    rb_dict = dict(zip(range(16),[.80,.80,.60,.30,0,0,0,0,0,0,0,0,0,0,0,0]))
    wr_dict = dict(zip(range(16),[.80,.80,.80,0.80,.30,0,0,0,0,0,0,0,0,0,0,0]))
    te_dict = dict(zip(range(16),[.80,.20,0,0,0,0,0,0,0,0,0,0,0,0,0,0]))
    dst_dict = dict(zip(range(16),[.93,.07,0,0,0,0,0,0,0,0,0,0,0,0,0,0]))
    k_dict = dict(zip(range(16),[.93,.07,0,0,0,0,0,0,0,0,0,0,0,0,0,0]))
    num_qb = sum(my_team['position']=='Q')
    num_rb = sum(my_team['position']=='R')
    num_wr = sum(my_team['position']=='W')
    num_te = sum(my_team['position']=='T')
    num_dst = sum(my_team['position']=='D')
    num_k = sum(my_team['position']=='K')
    
    # I also added a constraint to avoid drafting players at the same position with the same bye week
    qb_bye = list(my_team[my_team['position']=='Q']['bye'])
    rb_bye = list(my_team[my_team['position']=='R']['bye'])
    wr_bye = list(my_team[my_team['position']=='W']['bye'])
    te_bye = list(my_team[my_team['position']=='T']['bye'])
    dst_bye = list(my_team[my_team['position']=='D']['bye'])
    k_bye = list(my_team[my_team['position']=='K']['bye'])
    position_index = []

    # capturing start time so that I can calibrate how long the model takes for a specific number of simulations
    # so that I can ensure it runs in time on draft night
    start = timeit.default_timer()
    for i in range(len(picks)):
        for j in range((16-roundnum)*10):
            if j not in mypicks:
                bestnum = sorted(current_df['rank'].values)[random.randint(0,2)]
                bestname = current_df[current_df['rank'] == bestnum]['player'].values[0]
                current_df = dropplayer(current_df,bestname)
            else:
                # add light positional constraints just to prevent simulations that are unrealistic
                # to the point of not being informative
                if num_qb == 0:
                    position_index.append('Q')
                if ((num_qb == 1) and (roundnum >=9)):
                    position_index.append('Q')
                if num_rb < 3:
                    position_index.append('R')
                if ((num_rb == 3) and (roundnum >=9)):
                    position_index.append('R')
                if num_wr < 4:
                    position_index.append('W')
                if ((num_wr == 4) and (roundnum >=9)):
                    position_index.append('W')
                if num_te == 0:
                    position_index.append('T')
                if ((num_te == 1) and (roundnum >=9)):
                    position_index.append('T')
                if num_dst == 0:
                    position_index.append('D')
                if ((num_dst == 1) and (roundnum >=13)):
                    position_index.append('D')
                if num_k == 0:
                    position_index.append('K')
                if ((num_k == 1) and (roundnum >=13)):
                    position_index.append('K')
                randposition = position_index[random.randint(0,len(position_index)-1)]
                if randposition == 'Q':
                    k = 0
                    while(True):
                        if current_df[current_df['position']=='Q'].iloc[k,4] in qb_bye:
                            k += 1
                        else:
                            nextplayer = current_df[current_df['position']=='Q'].iloc[k,1]
                            position = 'Q'
                            currpoints = current_df[current_df['position']=='Q'].iloc[k,7] * qb_dict[num_qb]
                            num_qb += 1
                            qb_bye.append(current_df[current_df['position']=='Q'].iloc[k,4])
                            break
                elif randposition == 'R':
                    k = 0
                    while(True):
                        if current_df[current_df['position']=='R'].iloc[k,4] in rb_bye:
                            k += 1
                        else:
                            nextplayer = current_df[current_df['position']=='R'].iloc[k,1]
                            position = 'R'
                            currpoints = current_df[current_df['position']=='R'].iloc[k,7] * rb_dict[num_rb]
                            num_rb += 1
                            rb_bye.append(current_df[current_df['position']=='R'].iloc[k,4])
                            break
                elif randposition == 'W':
                    k = 0
                    while(True):
                        if current_df[current_df['position']=='W'].iloc[k,4] in wr_bye:
                            k += 1
                        else:
                            nextplayer = current_df[current_df['position']=='W'].iloc[k,1]
                            position = 'W'
                            currpoints = current_df[current_df['position']=='W'].iloc[k,7] * wr_dict[num_wr]
                            num_wr += 1
                            wr_bye.append(current_df[current_df['position']=='W'].iloc[k,4])
                            break
                elif randposition == 'T':
                    k = 0
                    while(True):
                        if current_df[current_df['position']=='T'].iloc[k,4] in te_bye:
                            k += 1
                        else:
                            nextplayer = current_df[current_df['position']=='T'].iloc[k,1]
                            position = 'T'
                            currpoints = current_df[current_df['position']=='T'].iloc[k,7] * te_dict[num_te]
                            num_te += 1
                            te_bye.append(current_df[current_df['position']=='T'].iloc[k,4])
                            break
                elif randposition == 'D':
                    k = 0
                    while(True):
                        if current_df[current_df['position']=='D'].iloc[k,4] in dst_bye:
                            k += 1
                        else:
                            nextplayer = current_df[current_df['position']=='D'].iloc[k,1]
                            position = 'DST'
                            currpoints = current_df[current_df['position']=='D'].iloc[k,7] * dst_dict[num_dst]
                            num_dst += 1
                            dst_bye.append(current_df[current_df['position']=='D'].iloc[k,4])
                            break
                else:
                    k = 0
                    while(True):
                        if current_df[current_df['position']=='K'].iloc[k,4] in k_bye:
                            k += 1
                        else:
                            nextplayer = current_df[current_df['position']=='K'].iloc[k,1]
                            position = 'K'
                            currpoints = current_df[current_df['position']=='K'].iloc[k,7] * k_dict[num_k]
                            num_k += 1
                            k_bye.append(current_df[current_df['position']=='K'].iloc[k,4])
                            break
                runpoints += currpoints
                picks.iloc[i,roundnum] = position
                picks.iloc[i,16] = runpoints
                roundnum += 1
                current_df = dropplayer(current_df,nextplayer)
                position_index = []
        roundnum = len(my_team)
        currpoints = 0
        runpoints = sum(my_team['points'])
        num_qb = sum(my_team['position']=='Q')
        num_rb = sum(my_team['position']=='R')
        num_wr = sum(my_team['position']=='W')
        num_te = sum(my_team['position']=='T')
        num_dst = sum(my_team['position']=='D')
        num_k = sum(my_team['position']=='K')
        qb_bye = list(my_team[my_team['position']=='Q']['bye'])
        rb_bye = list(my_team[my_team['position']=='R']['bye'])
        wr_bye = list(my_team[my_team['position']=='W']['bye'])
        te_bye = list(my_team[my_team['position']=='T']['bye'])
        dst_bye = list(my_team[my_team['position']=='D']['bye'])
        k_bye = list(my_team[my_team['position']=='K']['bye'])
        current_df = draft_df.copy()
    stop = timeit.default_timer()
    print('Time: ', stop - start)
    picks['Pts'] = picks['Pts'].astype(float)
    # Summary intended to show generally which position is the best one to choose in the next round
    return(picks.groupby(picks.columns[len(my_team)]).agg({'Pts':['count','min','mean','max']}).round())

### 4. Draft admin

Drop players who are drafted by other teams

In [55]:
print(len(draft_df))
draft_df = dropplayer(draft_df,"Dalvin Cook")
print(len(draft_df))

279
278


Run the draft simluation

In [56]:
simulate(my_team,draft_df,simulations=200)

Time:  93.98239949999993


Pts                        
     count     min    mean     max
Rnd1                              
DST     25  1307.0  1360.0  1418.0
K       40  1315.0  1367.0  1423.0
Q       35  1328.0  1399.0  1470.0
R       24  1333.0  1416.0  1520.0
T       41  1303.0  1400.0  1457.0
W       35  1323.0  1403.0  1478.0

Find the best available running back

In [57]:
findplayer(my_team,draft_df,'R')

,rank,player,position,team,bye,posrank,player_stat,points
6,7,Derrick Henry,R,TEN,7,6,Derrick Henry,239.81
4,5,Alvin Kamara,R,NO,6,5,Alvin Kamara,213.82
7,8,Clyde Edwards-Helaire,R,KC,10,7,Clyde Edwards-Helaire,203.83
10,11,Nick Chubb,R,CLE,9,10,Nick Chubb,203.61
9,10,Josh Jacobs,R,LV,6,9,Josh Jacobs,202.69


In [58]:
player = 'Derrick Henry'
my_team = addplayer(my_team,draft_df,player)
my_team.index = list(range(len(my_team)))
my_team

,rank,player,position,team,bye,posrank,player_stat,points
0,7,Derrick Henry,R,TEN,7,6,Derrick Henry,239.81


In [59]:
print(len(draft_df))
draft_df = dropplayer(draft_df,player)
print(len(draft_df))

278
277


### Conclusion

While the appoach might seem to conclude the obvious at points (e.g. not drafting a kicker in the first round), for me the value came in receiving guidance on some of the trickier tradeoffs in the draft.

Getting real-time feedback on whether to snag a tier 1 QB/TE early or wait for the mid rounds tier 2 was helpful.

Similarly, having feedback on how to balance RB vs WR in the opening rounds was also helpful.

It's also helpful for when a particular position starts flying off the board to know if you need to join the pack or can afford to hold off.

One last note - I also included a number of simulations parameter so that I could ensure the model would run in time on draft night.